In [13]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

Done.

nvidia.ko:
Running module version sanity check.
 - Original module
   - No original module exists within this kernel
 - Installation
   - Installing to /lib/modules/5.15.0-139-generic/updates/dkms/

nvidia-modeset.ko:
Running module version sanity check.
 - Original module
   - No original module exists within this kernel
 - Installation
   - Installing to /lib/modules/5.15.0-139-generic/updates/dkms/

nvidia-drm.ko:
Running module version sanity check.
 - Original module
   - No original module exists within this kernel
 - Installation
   - Installing to /lib/modules/5.15.0-139-generic/updates/dkms/

nvidia-uvm.ko:
Running module version sanity check.
 - Original module
   - No original module exists within this kernel
 - Installation
   - Installing to /lib/modules/5.15.0-139-generic/updates/dkms/

nvidia-peermem.ko:
Running module version sanity check.
 - Original module
   - No original module exists within this kernel
 - Installation
   - Installing to /lib/modules/5.15.0-1

In [15]:
# start server
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [16]:
# pull deepseek-r1:32b
!ollama pull deepseek-r1:1.5b

In [17]:
!pip install ollama

# New Section

In [18]:
import ollama
response = ollama.chat(model='deepseek-r1:1.5b', messages=[
  {
    'role': 'user',
    'content': "How many r's are in a strawberry?",
  },
])
print(response['message']['content'])

<think>
To determine how many 'r's are in the word "strawberry," I will start by examining each letter individually. 

First, the first letter is 's.' This does not spell an 'r,' so there are no 'r's here.

Next, the second letter is 't.' Again, this doesn't contain an 'r.'

The third letter is 'w,' which also doesn't meet the criteria.

Moving on to the fourth letter, it's 'a'—no 'r' here either.

The fifth letter is 'r,' which stands out. I will count this as one 'r.'

The sixth letter is 's.' This does not spell an 'r.'

Continuing with the seventh letter, it's 't,' so no 'r' there.

Finally, the eighth letter is 'e,' and since none of these letters are 'r,' I conclude that there are zero 'r's in "strawberry."
</think>

To determine how many times the letter **'r'** appears in the word "strawberry," let's examine each letter individually:

1. **s**: Not an 'r'.
2. **t**: Not an 'r'.
3. **w**: Not an 'r'.
4. **a**: Not an 'r'.
5. **r**: This is one 'r'.
6. **s**: Not an 'r'.
7. **t**

In [ ]:
import json
import requests, time


def readkey(key):
    # Read from colab user configuration first. if cannot find, read from local file
    ret=None
    #from google.colab import userdata
    #ret=userdata.get(key)
    
    if not ret:
        with open('config.json', 'r') as file:
            ret=json.load(file)
            print(f"{ret}")
    return ret


In [ ]:
import google.generativeai as genai
from google.colab import userdata
gemini_api=userdata.get("gemini")

q='describe singapore in 10 words'

# for local testing only
#gemini_api = readkey('gemini')

genai.configure(api_key=gemini_api)
geminimodel= genai.GenerativeModel("gemini-2.0-flash") #gemini-1.5-flash obsoleted


In [ ]:
# this block is to create models
# Stable diffusion model
import torch
from diffusers import StableDiffusionPipeline
stablemodel = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
#pipe = pipe.to("cuda")
#prompt = input("image : ")


In [ ]:

image = stablemodel('pug').images[0]
image

In [ ]:
# for local testing only
#telegram_api_key=readkey("telegram_key")

telegram_api_key=userdata.get("telegram_key")

url=f"https://api.telegram.org/bot{telegram_api_key}"

# Read from Telegram message queue
r=requests.get(url+"/getUpdates")
print(f"{r}")
r=r.json()
r=r['result']
chat_id=r[-1]['message']['chat']['id']



In [ ]:
# Create a loop routine to simulate the chatbot
firstmsgid=0
r=requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'type [question] with model selection [Gemin or DeepSeek or Stable] at latest or type [quit]'}")

while True:
    #while True:
    r1=requests.get(url+"/getUpdates")
    time.sleep(1)
    #print(r.json())
    r1=r1.json()
    r1=r1["result"][-1]['message']
    q=r1['text']
    msg_id=r1['message_id']

    if firstmsgid==0 or firstmsgid==msg_id:
      firstmsgid=msg_id
      print(f"firstid={firstmsgid},CHAT_ID={r1['chat']['id']}")
      continue
    elif firstmsgid<msg_id:
    #  print(f"B msg={q}")
      firstmsgid=msg_id
    #  break

    if q=='quit':
        print(f"msg={q}")
        break
    else:
      q=str.lower(q)
      qr=q.split(" ")
      modelstr=qr[-1]
      if modelstr=='stable':
        q=q.replace('stable',"")
        image = pipe(q).images[0]
        image.save("/content/img.png")
        image_caption="stable-diffusion"
        data={"chat_id":chat_id,"caption":image_caption}
        #r2=model.generate_content(q)
        url2=url+f"/sendPhoto?chat_id={chat_id}"
        with open("/content/img.png", "rb") as f:
          ret=requests.post(url2, files={"photo":f}, data=data)
          ret.json()

      else:
        if modelstr=='deepseek':
          q=q.replace('deepseek',"")
          response = ollama.chat(model='deepseek-r1:1.5b', messages=[
            {
              'role': 'user',
              'content': q,
            },
          ])
          r2=requests.get(url+f"/sendMessage?chat_id={chat_id}&text={response['message']['content']}")
        elif modelstr=='gemini':
          q.replace('gemini', "")
          r2=geminimodel.generate_content(q)
          r2=requests.get(url+f"/sendMessage?chat_id={chat_id}&text={r2.text}")
        else:
          messages = "Unknown model"

          r2=requests.get(url+f"/sendMessage?chat_id={chat_id}&text={messages}")
      r=requests.get(url+f"/sendMessage?chat_id={chat_id}&text={'type [question] with model selection [Gemin or DeepSeek or Stable] at latest or type [quit]'}")

print("End")